<a href="https://colab.research.google.com/github/rhehgus02/ESAA_OB/blob/main/%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


-> 전체 데이터셋을 한 번에 메모리에 로드하지 않고 미니 배치 학습을 사용할 수 있다. 또한 과적합을 방지하기 위해 정규화 기법을 사용하는 것이 중요하다. L1 또는 L2 정규화를 적용하여 모델을 안정화시킬 수 있다. 하지만 정규 방정식이나 SVD 방법은 계산 복잡도가 특성 개수에 따라 매우 빠르게 증가하기 때문에 사용할 수 없다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

-> 에포크마다 검증 에러가 지속적으로 상승한다면 한 가지 가능성은 학습률이 너무 높아 알고리즘이 발산하는 것일 수 있다. 훈련 에러도 올라간다면 이 문제가 확실하므로, 학습률을 낮추어야 한다. 하지만 훈련 에러가 증가하지 않는다면, 모델이 훈련 세트에 과대적합 되어 있는 것이므로 훈련을 멈춰야 한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

->
릿지 회귀에서 훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높다는 현상은 일반적으로 모델의 높은 편향을 나타내는 신호이다. 이런 경우에는 과소적합 문제가 있을 가능성이 높다. 과소적합은 모델이 데이터를 충분히 학습하지 못하거나 모델이 너무 단순하게 설정되었을 때 발생할 수 있다. 즉, 규제 하이퍼파라미터  α 를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

-> 규제가 있는 모델이 일반적으로 규제가 없는 모델보다 성능이 좋기 때문이다.
라쏘 회귀는  l1  페널티를 사용하여, 가중치를 완전히 0으로 만드는 경향이 있다. 이는 가장 중요한 가중치를 제외하고는 모두 0이 되는 희소한 모델을 만든다. 또한 자동으로 특성 선택의 효과를 가지므로, 일부 특성만 실제로 유용할 것 같을 때 사용하면 좋다. 확신이 없다면 릿지 회귀를 사용해야 한다.
라쏘가 어떤 경우에는 불규칙하게 행동하므로, 엘라스틱넷이 라쏘보다 일반적으로 선호된다. 그러나 추가적인 하이퍼 파라미터가 생긴다. 불규칙한 행동이 없는 라쏘를 원하면 엘라스틱 넷에 l1_ratio를 1에 가깝게 설정하면 좋다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [ ]:
import numpy as np

# 소프트맥스 함수 정의
def softmax(scores):
    exp_scores = np.exp(scores - np.max(scores, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

# 손실 함수 정의 (크로스 엔트로피)
def cross_entropy_loss(y, y_pred):
    m = y.shape[0]
    return -np.sum(y * np.log(y_pred)) / m

# 소프트맥스 회귀 모델 클래스 정의
class SoftmaxRegression:
    def __init__(self, input_dim, num_classes, learning_rate=0.01, max_epochs=100, early_stopping_rounds=10):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.early_stopping_rounds = early_stopping_rounds

    def fit(self, X, y, X_val=None, y_val=None):
        m, n = X.shape
        self.weights = np.zeros((n, self.num_classes))
        self.bias = np.zeros(self.num_classes)

        best_val_loss = np.inf
        early_stopping_count = 0

        for epoch in range(self.max_epochs):
            # 예측 계산
            scores = np.dot(X, self.weights) + self.bias
            y_pred = softmax(scores)

            # 손실 계산
            loss = cross_entropy_loss(y, y_pred)

            # 그래디언트 계산
            gradient = np.dot(X.T, (y_pred - y)) / m

            # 가중치 업데이트
            self.weights -= self.learning_rate * gradient

            # 조기 종료 검증
            if X_val is not None and y_val is not None:
                val_scores = np.dot(X_val, self.weights) + self.bias
                val_pred = softmax(val_scores)
                val_loss = cross_entropy_loss(y_val, val_pred)
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stopping_count = 0
                else:
                    early_stopping_count += 1
                    if early_stopping_count == self.early_stopping_rounds:
                        print(f"Early stopping at epoch {epoch} with validation loss {val_loss}")
                        break

            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss = {loss}")

    def predict(self, X):
        scores = np.dot(X, self.weights) + self.bias
        y_pred = softmax(scores)
        return np.argmax(y_pred, axis=1)